**Coin-Row problem**: Given a row of `n` coins, with values `[c1, c2, ..., cn]`, the goal is to pick up the subset of coins with the largest combined value subject to the constraint that we cannot pick any two adjacent coins.

Let `F(n)` be defines as the largest combined value given a row of `n` coins.

Using the dynamic programming approach, we consider two possible solutions:

(1) The solution subset includes the `nth` coin => it cannot contain the `(n-1)th` coin such that the remaining coins must be picked from the first `(n-2)` and the solution can be expressed as:

`F(n) = cn + F(n-2)`

(2) The solution subset does not contain the nth coin => the coins that can be picked must come from the first `(n-1)` coins, so the solution can be expressed as:

`F(n) = F(n-1)`


Then the final solution must be the larger of these two possibilities:

`F(n) = max(F(n-1), cn + F(n-2))`

which is a `recurrance relation` with base case `F(0) = 0, F(1) = c1`


In [36]:
import numpy as np

def coin_row(C):
    # create an array to store solutions for subproblems
    F = np.zeros(shape=(len(C)+1))

    # base cases
    F[0] = 0
    F[1] = C[0]

    # coins pick for each subproblem solution
    coins_picked = [[None], [C[0]]]
    
    # bottom up solution for finding F[n-1]
    for i in range(2,len(C)+1):
        F[i] = max(F[i-1],  C[i-1]+F[i-2])
        print(f"i = {i}, F[{i}] = {F[i]}, F[{i-1}] = {F[i-1]}, F[{i-2}] = {F[i-2]}, C[{i-1}] = {C[i-1]}")
        
        if(C[i-1]+F[i-2] > F[i-1]):
            coins_picked.append([C[i-1]]+coins_picked[i-2])
        else:
            coins_picked.append(coins_picked[i-1])   

        print(f"Coins picked: {coins_picked}")         

    return F, coins_picked

In [37]:

# row of 6 coins with the following values
C = np.array([5.,1.,2.,10.,6.,2.])

F, coins_picked = coin_row(C)



i = 2, F[2] = 5.0, F[1] = 5.0, F[0] = 0.0, C[1] = 1.0
Coins picked: [[None], [5.0], [5.0]]
i = 3, F[3] = 7.0, F[2] = 5.0, F[1] = 5.0, C[2] = 2.0
Coins picked: [[None], [5.0], [5.0], [2.0, 5.0]]
i = 4, F[4] = 15.0, F[3] = 7.0, F[2] = 5.0, C[3] = 10.0
Coins picked: [[None], [5.0], [5.0], [2.0, 5.0], [10.0, 5.0]]
i = 5, F[5] = 15.0, F[4] = 15.0, F[3] = 7.0, C[4] = 6.0
Coins picked: [[None], [5.0], [5.0], [2.0, 5.0], [10.0, 5.0], [10.0, 5.0]]
i = 6, F[6] = 17.0, F[5] = 15.0, F[4] = 15.0, C[5] = 2.0
Coins picked: [[None], [5.0], [5.0], [2.0, 5.0], [10.0, 5.0], [10.0, 5.0], [2.0, 10.0, 5.0]]


In [38]:
print(C)
print(F)
print(coins_picked)

[ 5.  1.  2. 10.  6.  2.]
[ 0.  5.  5.  7. 15. 15. 17.]
[[None], [5.0], [5.0], [2.0, 5.0], [10.0, 5.0], [10.0, 5.0], [2.0, 10.0, 5.0]]
